In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
# import gdown
# gdown.download(id='1viGXAYR5672DWm-iit1N90s6Uxwxj6qm')  #archive with indicies

In [40]:
# scannet indicies
## intrinsics.npz: (9, ) array for each scene (but in dataset they also exist)
## statistics.json: number of pairs? for 'train', 'val' 
## scene_data
### train_list: train-val sequences names
### train:  seq.npz: ['name']: array(N, 4) [space_id, scan_id, image1_id, image2_id] ['score']: array(N, )
### val:  seq.npz: array(N, 4) [space_id, scan_id, image1_id, image2_id]

In [2]:
import sys
sys.path.append('../src')

In [3]:
import json
import os
import numpy as np
from tqdm.auto import tqdm
import pandas as pd
import seaborn as sns
from queue import PriorityQueue
import shutil

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# V1

### Train subset

In [451]:
np.random.seed(42)

train_indicies_path = '../../ScanNet/scannet_indices/scene_data/train/'
r_compression = 1 / 2200

subset_train_indicies = None

for scene_npz in tqdm(os.listdir(train_indicies_path)):
    scene_data = np.load(train_indicies_path + scene_npz)
    
    covisibility_criteria = (scene_data['score'] >= 0.4) & (scene_data['score'] <= 0.8)
    scene_pairs = scene_data['name'][covisibility_criteria, :]
    scene_scores = scene_data['score'][covisibility_criteria]
    n_of_pairs = scene_pairs.shape[0]
        
    n_to_choose = int(n_of_pairs * r_compression)
    pairs_to_keep = np.random.choice(np.arange(n_of_pairs), n_to_choose, replace=False)
    
    if subset_train_indicies is None:
        subset_train_indicies = scene_pairs[pairs_to_keep, :]
        subset_train_scores = scene_scores[pairs_to_keep]
    else:
        subset_train_indicies = np.concatenate([subset_train_indicies, scene_pairs[pairs_to_keep, :]])
        subset_train_scores = np.append(subset_train_scores, scene_scores[pairs_to_keep])
        
np.savez('/home/project/ScanNet/train_indices_subset.npz', name=subset_train_indicies, score=subset_train_scores)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1513/1513 [00:51<00:00, 29.34it/s]


In [ ]:
len(subset_train_indicies)

99586

### Val subset

In [455]:
np.random.seed(42)

val_indicies_path = '../../ScanNet/scannet_indices/scene_data/val/'
r_compression = 0.1

subset_val_indicies = None
n_total = 0
for scene_npz in tqdm(os.listdir(val_indicies_path)):
    scene_data = np.load(val_indicies_path + scene_npz)
    
    scene_pairs = scene_data['name']
    scene_scores = scene_data['score']
    n_of_pairs = scene_pairs.shape[0]
    n_total += n_of_pairs
    n_to_choose = int(n_of_pairs * r_compression)
    pairs_to_keep = np.random.choice(np.arange(n_of_pairs), n_to_choose, replace=False)
    
    if subset_val_indicies is None:
        subset_val_indicies = scene_pairs[pairs_to_keep, :]
        subset_val_scores = scene_scores[pairs_to_keep]
    else:
        subset_val_indicies = np.concatenate([subset_val_indicies, scene_pairs[pairs_to_keep, :]])
        subset_val_scores = np.append(subset_val_scores, scene_scores[pairs_to_keep])
        
np.savez('/home/project/ScanNet/val_indices_subset.npz', name=subset_val_indicies, score=subset_val_scores)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1513/1513 [00:01<00:00, 1221.11it/s]


In [456]:
len(subset_val_indicies)

42262

### Create subset of ScanNet

In [9]:
# image_pairs_train = np.load('/home/project/ScanNet/train_indices_subset.npz')
# image_pairs_val = np.load('/home/project/ScanNet/val_indices_subset.npz')

In [11]:
# all_indicies = np.concatenate([image_pairs_train['name'], image_pairs_val['name']], axis=0)

In [12]:
# all_indicies.shape

(118820, 4)

In [13]:
# images = set()
# for s in all_indicies:
#     im1 = (s[0], s[1], s[2])
#     im2 = (s[0], s[1], s[3]) 
#     images.update([im1, im2])

In [26]:
# src_path = '/home/project/data/scans/'
# dst_path = '/home/project/data_sample/scans/'

# for image in tqdm(images):
#     scene_name = f'scene{image[0]:04d}_{image[1]:02d}/'
#     rgb_name = f'{image[2]}.jpg'
#     depth_name = f'{image[2]}.png'
#     pose_name = f'{image[2]}.txt'
    
#     shutil.copyfile(src_path+scene_name+'color/'+rgb_name, dst_path+scene_name+'color/'+rgb_name)
#     shutil.copyfile(src_path+scene_name+'depth/'+depth_name, dst_path+scene_name+'depth/'+depth_name)
#     shutil.copyfile(src_path+scene_name+'pose/'+pose_name, dst_path+scene_name+'pose/'+pose_name)

100%|█████████████████████████████████████████████████████████████████████████| 222123/222123 [1:52:33<00:00, 32.89it/s]


### Allign test data with loading protocol

In [29]:
scannet_test_pairs = pd.read_csv('../../ScanNet/scannet_test_pairs_with_gt.txt', sep='\s', header=None)

scannet_test_pairs['image0_id'] = scannet_test_pairs[0].apply(lambda x: int(x.split('/')[-1].split('-')[1].split('.')[0]))
scannet_test_pairs['image1_id'] = scannet_test_pairs[1].apply(lambda x: int(x.split('/')[-1].split('-')[1].split('.')[0]))
scannet_test_pairs['scene_name'] = scannet_test_pairs[0].apply(lambda x: int(x.split('/')[1].split('scene')[1].split('_')[0]))
scannet_test_pairs['scene_subname'] = scannet_test_pairs[0].apply(lambda x: int(x.split('/')[1].split('scene')[1].split('_')[1]))

np.savez('test_indices.npz', name=scannet_test_pairs[['scene_name', 'scene_subname', 'image0_id', 'image1_id']].values)

K_dict = {}
for seq in scannet_test_pairs[0].apply(lambda x: x.split('/')[1]).unique():
    K = np.loadtxt('../../ScanNet/scannet_test_1500/'+seq+'/intrinsic/intrinsic_depth.txt')[:3, :3].reshape(-1)
    K_dict[seq] = K

np.savez('intrinsics_test.npz', **K_dict)

/tmp/ipykernel_43467/937024916.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  scannet_test_pairs = pd.read_csv('../../ScanNet/scannet_test_pairs_with_gt.txt', sep='\s', header=None)


In [32]:
intr = np.load('../../ScanNet/scannet_indices/intrinsics.npz')

In [22]:
p0 = np.loadtxt('../../ScanNet/scannet_test_1500/scene0707_00/pose/15.txt')
p1 = np.loadtxt('../../ScanNet/scannet_test_1500/scene0707_00/pose/585.txt')

In [26]:
scannet_test_pairs.iloc[0][-16:].values.reshape(4, 4)

array([[0.97598, 0.05925, 0.20966, -0.03968],
       [-0.08203, 0.99143, 0.10171, 0.00746],
       [-0.20184, -0.11647, 0.97247, 0.54255],
       [0.0, 0.0, 0.0, 1.0]], dtype=object)

In [27]:
np.linalg.inv(p1) @ p0

array([[ 0.97597637,  0.05924605,  0.20966387, -0.03968865],
       [-0.08203433,  0.99142575,  0.10171394,  0.00745622],
       [-0.20184084, -0.11647032,  0.97246969,  0.54254895],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [9]:
test_data = ScanNetDataset(root_dir='scannet_test_1500/',
               npz_path='test.npz',
               intrinsics_path='intrinsics_test.npz')

In [202]:
np.load('scannet_indices/scene_data/train/scene0083_01.npz')['name']

array([[ 83,   1, 396, 488],
       [ 83,   1, 336, 353],
       [ 83,   1,  58, 209],
       ...,
       [ 83,   1,  57, 129],
       [ 83,   1,  62,  83],
       [ 83,   1, 358, 387]], dtype=uint16)

In [195]:
np.loadtxt('/home/project/data/scans/scene0000_01/intrinsic/intrinsic_depth.txt')

array([[577.590698,   0.      , 318.905426,   0.      ],
       [  0.      , 578.729797, 242.683609,   0.      ],
       [  0.      ,   0.      ,   1.      ,   0.      ],
       [  0.      ,   0.      ,   0.      ,   1.      ]])

In [196]:
intr  = np.load('scannet_indices/intrinsics.npz')

In [197]:
intr['scene0000_01']

array([577.5907 ,   0.     , 318.90543,   0.     , 578.7298 , 242.68361,
         0.     ,   0.     ,   1.     ])

In [186]:
n_of_train = 0
for seq in tqdm(os.listdir(PATH_TRAIN)):
    npz_seq = np.load(PATH_TRAIN + seq)
    n_of_train += ((npz_seq['score'] > 0.4) & (npz_seq['score'] < 0.8)).sum() 

100%|███████████████████████████████████████████████████████████████████████████████| 1513/1513 [00:18<00:00, 81.84it/s]


In [187]:
print('Number of images in train:',  n_of_train / 10 ** 6)

Number of images in train: 220.382338


In [190]:
n_of_val = 0
for seq in tqdm(os.listdir(PATH_VAL)):
    npz_seq = np.load(PATH_VAL + seq)
    n_of_val += len(npz_seq['score']) 

100%|█████████████████████████████████████████████████████████████████████████████| 1513/1513 [00:00<00:00, 2568.41it/s]


In [191]:
print('Number of images in validation:', n_of_val / 10 ** 6)

Number of images in validation: 0.429409


In [38]:
n_of_train /  10 ** 6

220.382338

In [35]:
n_of_train + n_of_val

220603378

In [84]:
# !ln -s /home/project/data/scans_test/* /home/project/LoFTR/data/scannet/test

In [86]:
# !ln -s /home/project/ScanNet/scannet_indices/* /home/project/LoFTR/data/scannet/index

# V2

# Smart stratification

Idea here is to keep the minimal number of image pairs from original dataset, but all images should be present in subset and stratification by covisibility level should be done.


In [440]:
def build_graph(pairs: np.ndarray, covisibility_scores: np.ndarray, covisibility_thresholds: np.ndarray):
    """
    Returns graph in format: {'node_1': {(neighbor_11, covisibility_bin, covisibility_score), ... (neighbor_1k, covisibility_bin, covisibility_score)}, ..., 'node_n': {...}}
    """
    graph = {}
    for i, pair in enumerate(pairs):
        
        node_1 = (pair[0], pair[1], pair[2])
        node_2 = (pair[0], pair[1], pair[3])
        score = covisibility_scores[i]
        bins = (score >= covisibility_thresholds[:, 0]) & (score < covisibility_thresholds[:, 1])
        bin_ = np.argmax(bins)
                
        if bins.sum():
            graph.setdefault(node_1, set()).update([(node_2, bin_, score)])
            graph.setdefault(node_2, set()).update([(node_1, bin_, score)])
    return graph
    

def build_queue(graph):
    queue = PriorityQueue()
    for root, neighbours in graph.items():
        neighbours = list(neighbours)
        N = len(neighbours)
        var = sum((x[2] ** 2 for x in neighbours)) / N - (sum((x[2] for x in neighbours)) / N) ** 2
        queue.put((var, (root, neighbours)))
    return queue
    
    

def smart_stratification(pairs: np.ndarray, covisibility_scores: np.ndarray,
                         covisibility_thresholds: np.ndarray, global_distribution: np.ndarray):
    graph = build_graph(pairs, covisibility_scores, covisibility_thresholds)
    queue = build_queue(graph)
        
    keep_pairs = []
    keep_scores = []
    taken_nodes = set()
    n_bins = len(global_distribution)
    
    
    while not queue.empty():
        _, (root, neighbours) = queue.get()
        if root not in taken_nodes:
            step_importances = dict(zip(np.argsort(-global_distribution), np.arange(n_bins)))
            neighbours_importances = []
            for neigh, bin_, score in neighbours:
                if neigh in taken_nodes:
                    neigh_importance = 0
                else:
                    neigh_importance = step_importances[bin_]
                neighbours_importances.append(neigh_importance)
                
            choice_idx = np.random.choice(np.where(neighbours_importances == np.max(neighbours_importances))[0])
            taken_nodes.update([root, neighbours[choice_idx][0]])
            keep_pairs.append((root[0], root[1], root[2], neighbours[choice_idx][0][2]))
            keep_scores.append(neighbours[choice_idx][2])
            global_distribution[neighbours[choice_idx][1]] += 1
            
    return keep_pairs, keep_scores, global_distribution

### On train

In [526]:
np.random.seed(42)

train_indicies_path = '../../ScanNet/scannet_indices/scene_data/train/'
ts = np.linspace(0.4, 0.8, 41)
covisibility_thresholds = np.concatenate([ts[:-1].reshape(-1, 1), ts[1:].reshape(-1, 1)], axis=1)
global_distribution = np.zeros(40)

all_pairs = []
all_scores = []

for scene_npz in tqdm(os.listdir(train_indicies_path)):
    scene_data = np.load(train_indicies_path + scene_npz)
    pairs, scores, global_distribution = smart_stratification(scene_data['name'], scene_data['score'], covisibility_thresholds, global_distribution)
    all_pairs.extend(pairs)
    all_scores.extend(scores)

all_pairs = np.array(all_pairs)
all_scores = np.array(all_scores)

# np.savez('/home/project/code/data/scannet_splits/smart_sample_train.npz', name=all_pairs, score=all_scores)    

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1513/1513 [1:48:10<00:00,  4.29s/it]


In [527]:
bins = np.argmax((all_scores[:, None] >= covisibility_thresholds[:, 0]) & (all_scores[:, None] < covisibility_thresholds[:, 1]), axis=1)
n_in_bin = 2500
keep_ids = np.empty(0)

for bin_ in range(40):
    ids = np.argwhere(bins == bin_).ravel()
    keep_ids = np.append(keep_ids, np.random.choice(ids, size=n_in_bin, replace=False))
    
keep_ids = keep_ids.astype('int')
pairs_subset = all_pairs[keep_ids, :]
scores_subset = all_scores[keep_ids]

np.savez('/home/project/code/data/scannet_splits/smart_sample_train_ft.npz', name=pairs_subset, score=scores_subset)    

### On val

In [524]:
np.random.seed(42)

val_indicies_path = '../../ScanNet/scannet_indices/scene_data/val/'
ts = np.linspace(0.4, 0.8, 41)
covisibility_thresholds = np.concatenate([ts[:-1].reshape(-1, 1), ts[1:].reshape(-1, 1)], axis=1)
global_distribution = np.zeros(40)

all_pairs = []
all_scores = []

for scene_npz in tqdm(os.listdir(val_indicies_path)):  
    scene_data = np.load(val_indicies_path + scene_npz)
    pairs, scores, global_distribution = smart_stratification(scene_data['name'], scene_data['score'], covisibility_thresholds, global_distribution)
    all_scores.extend(scores)
    all_pairs.extend(pairs)
    
all_pairs = np.array(all_pairs)
all_scores = np.array(all_scores)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1513/1513 [00:13<00:00, 109.21it/s]


In [525]:
bins = np.argmax((all_scores[:, None] >= covisibility_thresholds[:, 0]) & (all_scores[:, None] < covisibility_thresholds[:, 1]), axis=1)
n_in_bin = 1000
keep_ids = np.empty(0)

for bin_ in range(40):
    ids = np.argwhere(bins == bin_).ravel()
    keep_ids = np.append(keep_ids, np.random.choice(ids, size=n_in_bin, replace=False))
    
keep_ids = keep_ids.astype('int')
pairs_subset = all_pairs[keep_ids, :]
scores_subset = all_scores[keep_ids]

np.savez('/home/project/code/data/scannet_splits/smart_sample_val.npz', name=pairs_subset, score=scores_subset)    